In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#using the requests and beautifulsoup libraries to extract the table from Wikipedia for futher manipulations
url = 'https://en.wikipedia.org/wiki/List_of_cities_in_Los_Angeles_County,_California'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
my_table = soup.find('table',{'class':'wikitable sortable'})

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [3]:
import re

def process_num(num):
    return int(re.sub(r'[^\w\s]','',num))

links = my_table.findAll('td')
a =[]
for link in links:
    a.append(link)

#creating three lists to separate the values that we extracted from wikipedia
City = []
Date_Inc = []
Population = []

#since every three items in the list "a" constitute one row in the table, we use a for loop to designate values into the three lists
for i in range(0,len(a),3):
    pc = a[i]
    City.append(pc.text.strip())
            
    bor = a[i+1]
    Date_Inc.append(bor.text.strip())
            
    neigh = a[i+2]
    Population.append(process_num(neigh.text.strip()))

#creating a dataframe using the three lists with indexes "Postal_Code", "Borough", and "Neighborhood"
data = {'City':City, 'Date_Inc':Date_Inc, 'Population':Population} 
table = pd.DataFrame(data)
table.shape

(88, 3)

In [4]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    g = geocoder.arcgis('{}, Los Angeles, United States'.format(neighborhood))
    lat_lng_coords = g.latlng
    return lat_lng_coords

In [5]:
!conda install -c conda-forge geocoder --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [6]:
import geocoder

coords = [get_latlng(neighborhood) for neighborhood in table["City"].tolist()]

In [7]:
coords_df = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
table['Latitude'] = coords_df["Latitude"]
table['Longitude'] = coords_df["Longitude"]
table.drop('Date_Inc',axis=1).head(5)
table.head(5)

,City,Date_Inc,Population,Latitude,Longitude
0,Agoura Hills,"December 8, 1982",20330,34.14611,-118.77812
1,Alhambra,"July 11, 1903",83653,34.09370,-118.12727
2,Arcadia,"August 5, 1903",56364,34.13635,-118.03867
3,Artesia,"May 29, 1959",16522,33.86114,-118.07968
4,Avalon,"June 26, 1913",3728,33.34411,-118.32139


In [23]:
CLIENT_ID = 'QPBRZHCQQ3KXFBZMEU1KFP1PH5FLEI1KBCLU0ZD5RT1JTMZT' # your Foursquare ID
CLIENT_SECRET = 'E2KIJBMDFJCCM3TBJLUVOYAWAVVC0XDE0IU5TKMECOPUUD4C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 2000

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: QPBRZHCQQ3KXFBZMEU1KFP1PH5FLEI1KBCLU0ZD5RT1JTMZT
CLIENT_SECRET:E2KIJBMDFJCCM3TBJLUVOYAWAVVC0XDE0IU5TKMECOPUUD4C


In [24]:
LA_venues = getNearbyVenues(names=table['City'],
                                   latitudes=table['Latitude'],
                                   longitudes=table['Longitude']
                                  )

In [25]:
LA_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agoura Hills,34.14611,-118.77812,Future Track Running Center,34.145819,-118.779251,Sporting Goods Shop
1,Agoura Hills,34.14611,-118.77812,Twisted Oak Tavern,34.145308,-118.778679,Gastropub
2,Agoura Hills,34.14611,-118.77812,Cafe Bizou,34.148410,-118.782587,French Restaurant
3,Agoura Hills,34.14611,-118.77812,Grissini Ristorante,34.145815,-118.778534,Italian Restaurant
4,Agoura Hills,34.14611,-118.77812,Target,34.148045,-118.793964,Big Box Store


In [26]:
LA_venues.groupby(["Neighborhood"]).count().head(5)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agoura Hills,98,98,98,98,98,98
Alhambra,100,100,100,100,100,100
Arcadia,100,100,100,100,100,100
Artesia,100,100,100,100,100,100
Avalon,72,72,72,72,72,72


In [27]:
print('There are {} uniques categories.'.format(len(LA_venues['Venue Category'].unique())))

There are 345 uniques categories.


In [28]:
# one hot encoding
LA_onehot = pd.get_dummies(LA_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
LA_onehot['Neighborhoods'] = LA_venues['Neighborhood'] 

#move neighborhood column to the first column
fixed_columns = [LA_onehot.columns[-1]] + list(LA_onehot.columns[:-1])
LA_onehot = LA_onehot[fixed_columns]

print(LA_onehot.shape)
LA_onehot.head(5)

(7478, 346)


,Neighborhoods,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,...,Warehouse Store,Water Park,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agoura Hills,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agoura Hills,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agoura Hills,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Agoura Hills,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Agoura Hills,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
LA_onehot = LA_onehot.groupby(["Neighborhoods"]).mean()
filtered_df = LA_onehot[["Coffee Shop"]]
table = table.set_index(filtered_df.index)
filtered_df['Population'] = table["Population"]
filtered_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Coffee Shop,Population
Neighborhoods,,
Agoura Hills,0.020408,20330
Alhambra,0.010000,83653
Arcadia,0.020000,56364
Artesia,0.030000,16522
Avalon,0.013889,3728
Azusa,0.067416,46361
Baldwin Park,0.011364,75390
Bell,0.022222,35477
Bell Gardens,0.027027,42072


In [53]:
filtered_df.sort_values(by = ["Coffee Shop"], ascending = False, inplace = True)
filtered_df.reset_index(inplace = True)
filtered_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighborhoods,Coffee Shop,Population
0,Hidden Hills,0.092308,1856
1,Calabasas,0.085714,23058
2,Sierra Madre,0.079365,10917
3,Malibu,0.071429,12645
4,Norwalk,0.070423,105549
5,South Pasadena,0.070000,25619
6,Downey,0.070000,111772
7,Pasadena,0.070000,137122
8,Culver City,0.070000,38883
9,Azusa,0.067416,46361


In [54]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

LA_grouped_clustering = filtered_df.drop('Neighborhoods', 1)

range_n_clusters = [2,3,4,5,6,7,8,9,10]

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(LA_grouped_clustering)
    centers = clusterer.cluster_centers_

    score = silhouette_score(LA_grouped_clustering, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))

For n_clusters = 2, silhouette score is 0.9733071171405602)
For n_clusters = 3, silhouette score is 0.6571013687640082)
For n_clusters = 4, silhouette score is 0.6388710129273502)
For n_clusters = 5, silhouette score is 0.593483463132585)
For n_clusters = 6, silhouette score is 0.5766012331150155)
For n_clusters = 7, silhouette score is 0.5912956399037109)
For n_clusters = 8, silhouette score is 0.569504045181539)
For n_clusters = 9, silhouette score is 0.5712668807862603)
For n_clusters = 10, silhouette score is 0.5889596636468565)


In [60]:
kclusters = 3

LA_grouped_clustering = filtered_df.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(LA_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
      dtype=int32)

In [103]:
#table.reset_index()
LA_merged = filtered_df.copy()
LA_merged["Clusters"] = kmeans.labels_
merged_df = pd.merge(LA_merged, table, on='Neighborhoods')
merged_df.drop(["Population_y","City","Date_Inc"],1, inplace=True)

In [86]:
!conda install -c conda-forge folium --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.11.0              |             py_0          61 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          87 KB

The following NEW packages will be INSTALLED:

    branca: 0.4.1-py_0  conda-forge
    folium: 0.11.0-py_0 conda-forge


folium-0.11.0        | 61 KB     | ##################################### | 100% 
branca-0.4.1         | 26 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [99]:
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Libraries imported.")

Libraries imported.


In [100]:
# create map
latitude = '35.0522'
longitude = '-118.2437'

venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Neighborhoods'], merged_df['Clusters']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [107]:
merged_df[merged_df["Clusters"] == 0]

,Neighborhoods,Coffee Shop,Population_x,Clusters,Latitude,Longitude
7,Pasadena,0.070000,137122,0,34.14745,-118.14427
12,Long Beach,0.060000,462257,0,33.76672,-118.19240
18,Santa Clarita,0.060000,210888,0,34.41387,-118.55121
31,Palmdale,0.047619,152750,0,34.57936,-118.11659
34,Lancaster,0.043478,160316,0,34.69890,-118.14478
37,Torrance,0.040000,145438,0,33.83705,-118.34312
44,Glendale,0.040000,203054,0,34.14633,-118.24864
47,Pomona,0.039474,149058,0,34.05499,-117.75004


In [108]:
merged_df[merged_df["Clusters"] == 1]

,Neighborhoods,Coffee Shop,Population_x,Clusters,Latitude,Longitude
13,Los Angeles,0.06,3792621,1,34.05361,-118.2455


In [109]:
merged_df[merged_df["Clusters"] == 2]

,Neighborhoods,Coffee Shop,Population_x,Clusters,Latitude,Longitude
0,Hidden Hills,0.092308,1856,2,34.15918,-118.64025
1,Calabasas,0.085714,23058,2,34.15778,-118.63842
2,Sierra Madre,0.079365,10917,2,34.16187,-118.05716
3,Malibu,0.071429,12645,2,34.03652,-118.68796
4,Norwalk,0.070423,105549,2,33.91681,-118.07252
5,South Pasadena,0.070000,25619,2,34.11582,-118.15176
6,Downey,0.070000,111772,2,33.94077,-118.12768
8,Culver City,0.070000,38883,2,34.02161,-118.39621
9,Azusa,0.067416,46361,2,34.13361,-117.90589
10,Rancho Palos Verdes,0.065217,41643,2,33.74526,-118.40378
